In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

df = pd.read_csv('../data/processed/lol_matches_expanded.csv')
df_p = pd.read_csv('../data/raw/Worlds 2025 Main Event - Player Stats - OraclesElixir.csv')
df_c = pd.read_csv('../data/raw/Worlds 2025 Main Event - Champion Stats - OraclesElixir.csv')

In [6]:
# ============================================================================
# Champion Win Rate & Pick Rate Analysis
# ============================================================================

df_champ = df_c[['Champion', 'Pos', 'W%', 'P%', 'B%', 'CTR%']].copy()
df_champ = df_champ.replace('-', '0')
print("\n" + "=" * 80)
print("INDIVIDUAL CHAMPION PERFORMANCE")
print("=" * 80)

# Clean and prepare the data

df_champ['W%'] = df_champ['W%'].str.rstrip('%').astype('float') / 100
df_champ['P%'] = df_champ['P%'].str.rstrip('%').astype('float') / 100
df_champ['B%'] = df_champ['B%'].str.rstrip('%').astype('float') / 100
df_champ['CTR%'] = df_champ['CTR%'].str.rstrip('%').astype('float') / 100

# Calculate total games (approximately from pick rate)
total_games = len(df)
df_champ['Estimated_Picks'] = (df_champ['P%'] * total_games).round().astype(int)
df_champ['Estimated_Bans'] = (df_champ['B%'] * total_games).round().astype(int)
df_champ['Presence'] = df_champ['Estimated_Picks'] + df_champ['Estimated_Bans']

# Use actual GP (Games Played) if available
if 'GP' in df_champ.columns:
    df_champ['Picks'] = df_champ['GP']
else:
    df_champ['Picks'] = df_champ['Estimated_Picks']

# Create a clean champion stats dataframe
champ_df = df_champ[['Champion', 'Pos', 'Picks', 'W%', 'P%', 'B%', 'CTR%', 'Presence']].copy()
champ_df.columns = ['Champion', 'Position', 'Picks', 'Win_Rate', 'Pick_Rate', 'Ban_Rate', 'Counter_Pick_Rate', 'Presence']
champ_df = champ_df.sort_values('Presence', ascending=False)

print("\n🏆 TOP 10 MOST CONTESTED CHAMPIONS (Picks + Bans)")
print("-" * 80)
print(champ_df.head(10)[['Champion', 'Position', 'Picks', 'Win_Rate', 'Pick_Rate', 'Ban_Rate', 'Presence']].to_string(index=False))

print("\n⭐ TOP 10 HIGHEST WIN RATE CHAMPIONS (Min 5 picks)")
print("-" * 80)
high_wr = champ_df[champ_df['Picks'] >= 5].sort_values('Win_Rate', ascending=False)
print(high_wr.head(10)[['Champion', 'Position', 'Picks', 'Win_Rate']].to_string(index=False))

print("\n🚫 TOP 10 MOST BANNED CHAMPIONS")
print("-" * 80)
most_banned = champ_df.sort_values('Ban_Rate', ascending=False)
print(most_banned.head(10)[['Champion', 'Position', 'Ban_Rate', 'Pick_Rate', 'Win_Rate']].to_string(index=False))



INDIVIDUAL CHAMPION PERFORMANCE

🏆 TOP 10 MOST CONTESTED CHAMPIONS (Picks + Bans)
--------------------------------------------------------------------------------
Champion Position  Picks  Win_Rate  Pick_Rate  Ban_Rate  Presence
  Yunara      ADC     15      0.67      0.188     0.700        71
    Azir   Middle     14      0.50      0.175     0.637        65
 Orianna   Middle     19      0.58      0.238     0.563        64
   Neeko  Support     19      0.63      0.238     0.450        55
    Sion      Top     21      0.67      0.263     0.388        52
   Poppy  Support      8      0.88      0.100     0.538        51
 Alistar  Support     24      0.54      0.300     0.313        49
  Wukong   Jungle     21      0.57      0.263     0.350        49
   Poppy      Top      3      0.67      0.038     0.538        46
   Poppy   Jungle      2      0.50      0.025     0.538        45

⭐ TOP 10 HIGHEST WIN RATE CHAMPIONS (Min 5 picks)
-----------------------------------------------------------

In [9]:

# ============================================================================
# Champion Synergy Matrix using Co-occurrence
# ============================================================================

print("\n" + "=" * 80)
print("CHAMPION SYNERGY MATRIX")
print("=" * 80)

def calculate_synergy_matrix(df, role1='ADC', role2='Support'):
    """Calculate champion synergy based on win rates when played together"""
    
    # Get all champions in these roles
    champs_role1 = set()
    champs_role2 = set()
    
    for team in ['Blue', 'Red']:
        champs_role1.update(df[f'{team}_Pick_{role1}'].unique())
        champs_role2.update(df[f'{team}_Pick_{role2}'].unique())
    
    # Create synergy matrix
    synergy_data = defaultdict(lambda: defaultdict(lambda: {'wins': 0, 'games': 0}))
    
    for team in ['Blue', 'Red']:
        for idx, row in df.iterrows():
            c1 = row[f'{team}_Pick_{role1}']
            c2 = row[f'{team}_Pick_{role2}']
            won = row['Winner'] == row[f'{team}_Team']
            
            synergy_data[c1][c2]['games'] += 1
            if won:
                synergy_data[c1][c2]['wins'] += 1
    
    # Find best synergies
    synergies = []
    for c1 in synergy_data:
        for c2 in synergy_data[c1]:
            stats = synergy_data[c1][c2]
            if stats['games'] >= 2:  # Lower threshold for small dataset
                wr = stats['wins'] / stats['games']
                synergies.append({
                    role1: c1,
                    role2: c2,
                    'Games': stats['games'],
                    'Win_Rate': wr
                })
    
    return pd.DataFrame(synergies).sort_values('Win_Rate', ascending=False)

bot_synergy = calculate_synergy_matrix(df, 'ADC', 'Support')
m_jg_synergy = calculate_synergy_matrix(df, 'Jungle', 'Mid')
print("\n🔗 TOP ADC-SUPPORT SYNERGIES (Min 2 games)")
print("-" * 80)
if len(bot_synergy) > 0:
    for idx, row in bot_synergy.head(15).iterrows():
        print(f"{row['ADC']:<15} + {row['Support']:<15} | {row['Games']} games | {row['Win_Rate']:.1%} WR")

print("\n🔗 TOP JUNGLE-MID SYNERGIES (Min 2 games)")
print("-" * 80)
if len(m_jg_synergy) > 0:
    for idx, row in m_jg_synergy.head(15).iterrows():
        print(f"{row['Jungle']:<15} + {row['Mid']:<15} | {row['Games']} games | {row['Win_Rate']:.1%} WR")


CHAMPION SYNERGY MATRIX

🔗 TOP ADC-SUPPORT SYNERGIES (Min 2 games)
--------------------------------------------------------------------------------
Corki           + Poppy           | 3 games | 100.0% WR
Ezreal          + Neeko           | 2 games | 100.0% WR
Corki           + Alistar         | 3 games | 100.0% WR
Varus           + Braum           | 2 games | 100.0% WR
Varus           + Poppy           | 2 games | 100.0% WR
Varus           + Rakan           | 2 games | 100.0% WR
Kalista         + Renata Glasc    | 2 games | 100.0% WR
Yunara          + Bard            | 2 games | 100.0% WR
Yunara          + Alistar         | 3 games | 100.0% WR
Corki           + Leona           | 4 games | 75.0% WR
Yunara          + Neeko           | 3 games | 66.7% WR
Kai'Sa          + Neeko           | 5 games | 60.0% WR
Caitlyn         + Neeko           | 5 games | 60.0% WR
Caitlyn         + Alistar         | 2 games | 50.0% WR
Sivir           + Leona           | 2 games | 50.0% WR

🔗 TOP JUNGLE-MID

In [10]:
# ============================================================================
# Advanced Analysis - Counter Picks
# ============================================================================

print("\n" + "=" * 80)
print("PART 5: COUNTER MATCHUP ANALYSIS")
print("=" * 80)

def analyze_matchups(df, role='Mid'):
    """Analyze which champions counter others"""
    
    matchups = defaultdict(lambda: defaultdict(lambda: {'wins': 0, 'games': 0}))
    
    for idx, row in df.iterrows():
        blue_champ = row[f'Blue_Pick_{role}']
        red_champ = row[f'Red_Pick_{role}']
        blue_won = row['Winner'] == row['Blue_Team']
        
        # Blue vs Red
        matchups[blue_champ][red_champ]['games'] += 1
        if blue_won:
            matchups[blue_champ][red_champ]['wins'] += 1
        
        # Red vs Blue
        matchups[red_champ][blue_champ]['games'] += 1
        if not blue_won:
            matchups[red_champ][blue_champ]['wins'] += 1
    
    # Find strongest counters
    counters = []
    for champ in matchups:
        for opponent in matchups[champ]:
            stats = matchups[champ][opponent]
            if stats['games'] >= 2:
                wr = stats['wins'] / stats['games']
                counters.append({
                    'Champion': champ,
                    'vs': opponent,
                    'Games': stats['games'],
                    'Win_Rate': wr
                })
    
    return pd.DataFrame(counters).sort_values('Win_Rate', ascending=False)

mid_counters = analyze_matchups(df, 'Mid')
jungle_counters = analyze_matchups(df, 'Jungle')
top_counters = analyze_matchups(df, 'Top')
print("\n⚔️ TOP TOP LANE MATCHUPS (Min 2 games)")
print("-" * 80)
print("Champion        vs                 | Games   | Win Rate")
if len(top_counters) > 0:
    for idx, row in top_counters.head(15).iterrows():
        print(f"{row['Champion']:<15} vs {row['vs']:<15} | {row['Games']} games | {row['Win_Rate']:.1%} WR")

print("\n⚔️ TOP JUNGLE LANE MATCHUPS (Min 2 games)")
print("-" * 80)
print("Champion        vs                 | Games   | Win Rate")
if len(jungle_counters) > 0:
    for idx, row in jungle_counters.head(15).iterrows():
        print(f"{row['Champion']:<15} vs {row['vs']:<15} | {row['Games']} games | {row['Win_Rate']:.1%} WR")

print("\n⚔️ TOP MID LANE MATCHUPS (Min 2 games)")
print("-" * 80)
print("Champion        vs                 | Games   | Win Rate")
if len(mid_counters) > 0:
    for idx, row in mid_counters.head(15).iterrows():
        print(f"{row['Champion']:<15} vs {row['vs']:<15} | {row['Games']} games | {row['Win_Rate']:.1%} WR")


PART 5: COUNTER MATCHUP ANALYSIS

⚔️ TOP TOP LANE MATCHUPS (Min 2 games)
--------------------------------------------------------------------------------
Champion        vs                 | Games   | Win Rate
Poppy           vs K'Sante         | 2 games | 100.0% WR
Rek'Sai         vs Renekton        | 2 games | 100.0% WR
Rumble          vs K'Sante         | 5 games | 100.0% WR
Sion            vs K'Sante         | 2 games | 100.0% WR
Sion            vs Ambessa         | 5 games | 100.0% WR
Rumble          vs Sion            | 2 games | 100.0% WR
Galio           vs Rumble          | 3 games | 100.0% WR
Ambessa         vs Rumble          | 5 games | 80.0% WR
K'Sante         vs Mordekaiser     | 3 games | 66.7% WR
Rek'Sai         vs K'Sante         | 3 games | 66.7% WR
Ornn            vs Rumble          | 5 games | 60.0% WR
Renekton        vs K'Sante         | 5 games | 60.0% WR
Sion            vs Rek'Sai         | 7 games | 57.1% WR
Aatrox          vs Renekton        | 2 games | 50.0% W

In [14]:
# save the results to CSV files
champ_df.to_csv('../data/processed/champion_performance_summary.csv', index=False)
bot_synergy.to_csv('../data/processed/bot_synergy.csv', index=False)
m_jg_synergy.to_csv('../data/processed/mid_jungle_synergy.csv', index=False)
mid_counters.to_csv('../data/processed/mid_counters.csv', index=False)
jungle_counters.to_csv('../data/processed/jungle_counters.csv', index=False)
top_counters.to_csv('../data/processed/top_counters.csv', index=False,)